In [6]:
import pandas as pd
from pyvi import ViTokenizer, ViPosTagger
import nltk
import numpy as np

In [60]:
# filename = "./dataset/bạn bè.txt"
# df = pd.read_fwf(filename,header = None, encoding="utf-8")
filename =["bạn bè.txt","các câu hỏi phức tạp.txt","du lịch.txt","gia đình.txt","giải trí.txt",
           "học tập.txt","nghề nghiệp.txt","nghỉ lễ.txt","người yêu.txt","robot.txt","shoping.txt",
           "sở thích.txt","tdtu.txt","thông tin cá nhân.txt","trò chuyện về đi ăn.txt","tán gẫu.txt","đất nước.txt","địa chỉ.txt"]
# df = pd.read_fwf(filename,header = None)
# df.head()
print(len(filename))

18


In [61]:
df = df[0]

In [62]:
temp_ques = []
temp_ans = []
tag = []
for k in range(len(filename)):
    with open('dataset/' + filename[k], encoding="utf-8") as f:
        lines = f.readlines()
    for i in range(len(lines)):
        if lines[i].startswith('__eou__'):
            continue
        else:
            part = lines[i].strip('__eou__').split('__eou__')
            temp_ques.append(ViTokenizer.tokenize(part[0].lower().strip()))
            temp_ans.append(ViTokenizer.tokenize(part[1].lower().strip()))
            tag.append(filename[k].split(".")[0])

In [63]:
data = pd.DataFrame({'Question':temp_ques,'Answer':temp_ans,'Tag':tag})
data.head()

,Question,Answer,Tag
0,thích đánh_lộn không ?,ngon nhà_vô,bạn bè
1,solo yasua không,chấp lun 2 mạng đầu,bạn bè
2,mai đi picnic không ?,mai bận học rồi,bạn bè
3,mai học ca mấy vậy ?,mai học ca 3,bạn bè
4,còn tiền không ?,còn chết liền,bạn bè


In [64]:
ques = np.array(data["Question"])
ans = np.array(data["Answer"])
ques_sentences=[nltk.word_tokenize(sent) for sent in ques]
ans_sentences=[nltk.word_tokenize(sent) for sent in ans] 

In [65]:
def clean_sentences(sentences):
    Punc = {'.', ',', '...', '-', '“', '”', ':', '(', ')', '"', '!', '&', ';', '?', '*', ']', '>', '…', '’',"``","''"}
    clean_sent = []
    for sent in sentences:
        temp =[]
        for item in sent:
            if item in Punc:
                continue
            elif(item.isdigit()):
                temp.append("Num")
            else:
                temp.append(item)
        clean_sent.append(temp)
    return clean_sent

In [66]:
def clean_one_sentence(sentences):
    Punc = {'.', ',', '...', '-', '“', '”', ':', '(', ')', '"', '!', '&', ';', '?', '*', ']', '>', '…', '’',"``","''"}
    clean_sent = []
    for sent in sentences:
        if sent in Punc:
            continue
        elif(sent.isdigit()):
            clean_sent.append("Num")
        else:
            clean_sent.append(sent)
    return clean_sent

In [67]:
clean_ques = clean_sentences(ques_sentences)
clean_ans = clean_sentences(ans_sentences)
# print(len(clean_ques))
# print(len(clean_ans))

### Word2Vec

In [68]:
from gensim.models import Word2Vec
model = Word2Vec(clean_ques, min_count = 1,size = 100)

In [69]:
def convert2vec(words):
    _sum = np.array([0]*100)
    for word in words:
        if not(word in model.wv.vocab):
            continue
        vec = model.wv[word]
        _sum = _sum + vec
    return _sum/len(words)

In [70]:
X = []
for sent in clean_ques:
    X.append(convert2vec(sent))
X = np.array(X)

c:\python\python36\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in true_divide
  


In [71]:
from scipy import spatial

#Find the nearest vector to input question
def get_answer(input_sent):
    # Clean input_sent
    input_sent = ViTokenizer.tokenize(input_sent).lower().strip()
    input_sent = clean_one_sentence(nltk.word_tokenize(input_sent))
    
    #Turn input sentence into vector
    vector_input = convert2vec(input_sent)
    
    #Find the nearest question in train data | cosine similarity
    cosin_lst = []
    for idx, vector_train in enumerate(X):
        cosin_lst.append([1 - spatial.distance.cosine(vector_input, vector_train),idx])
    possion = max(cosin_lst)[1]
    print(ans[possion])

In [73]:
get_answer("solo yasou không?")

chấp lun 2 mạng đầu


## Counter

In [69]:
import math
from collections import Counter

In [70]:
def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x]**2 for x in vec1.keys()])
    sum2 = sum([vec2[x]**2 for x in vec2.keys()])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator

In [134]:
def chat(input_):
    x = ViTokenizer.tokenize(input_).lower().strip()
    text1 = clean_one_sentence(nltk.word_tokenize(x))
    vector1 = Counter(text1)
    cosin = []
    # temp = []

    for i in range(len(clean_ques)):
        text2 = clean_ques[i]
        vector2 = Counter(text2)
        cosin.append([get_cosine(vector1, vector2),i])
    possion = max(cosin)[1]
    print(ans[possion])

In [135]:
chat("Dạo này, Ổn không?>>")

buồn lắm bạn à , crush unfriend trên facebook tui rồi , giờ tôi muốn nhắn_tin với crush cũng k được nữa


In [ ]:
temp = "dữ liệu chatbot - dự án cuối kỳ"